In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
from timeit import default_timer as timer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import requests
import warnings
from IPython.display import HTML
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
df=pd.read_csv('iphone_purchase_records.csv')
df.head()

,Gender,Age,Salary,Purchase Iphone
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           400 non-null    object
 1   Age              400 non-null    int64 
 2   Salary           400 non-null    int64 
 3   Purchase Iphone  400 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 12.6+ KB


In [6]:
df.describe()

,Age,Salary,Purchase Iphone
count,400.000000,400.000000,400.000000
mean,37.655000,69742.500000,0.357500
std,10.482877,34096.960282,0.479864
min,18.000000,15000.000000,0.000000
25%,29.750000,43000.000000,0.000000
50%,37.000000,70000.000000,0.000000
75%,46.000000,88000.000000,1.000000
max,60.000000,150000.000000,1.000000


In [7]:
df.columns

Index(['Gender', 'Age', 'Salary', 'Purchase Iphone'], dtype='object')

In [8]:
#change the data type of the gender column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])

In [9]:
#Assigning x and y
x=df[['Gender','Age','Salary']]
y=df['Purchase Iphone']

In [10]:
x

,Gender,Age,Salary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


In [11]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchase Iphone, Length: 400, dtype: int64

In [12]:
#split the data to train and test
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [13]:
x_train

,Gender,Age,Salary
211,1,52,150000
101,1,28,59000
333,1,40,65000
128,1,30,17000
194,1,28,89000
...,...,...,...
10,0,26,80000
355,1,60,34000
40,0,27,17000
357,0,41,72000


In [14]:
y_train

211    1
101    0
333    0
128    0
194    0
      ..
10     0
355    1
40     0
357    0
161    0
Name: Purchase Iphone, Length: 300, dtype: int64

In [15]:
x_test

,Gender,Age,Salary
168,1,29,148000
342,0,38,65000
179,0,31,34000
118,1,40,59000
143,1,30,89000
...,...,...,...
272,1,60,42000
153,0,36,50000
254,0,50,44000
366,0,58,47000


In [16]:
y_test

168    1
342    0
179    0
118    0
143    0
      ..
272    1
153    0
254    0
366    1
217    0
Name: Purchase Iphone, Length: 100, dtype: int64

In [18]:
#patch with intel oneapi with scikit learn
#patch the scikit learn 
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [19]:
from sklearn.linear_model import LogisticRegression

params = {
    'C': 0.1,
    'solver': 'lbfgs',
    'multi_class': 'multinomial',
    'n_jobs': -1,
}
start = timer()
classifier = LogisticRegression(**params).fit(x_train, y_train)
train_patched = timer() - start
f"Intel® extension for Scikit-learn time: {train_patched:.2f} s"

'Intel® extension for Scikit-learn time: 0.45 s'

In [20]:
y_predict = classifier.predict_proba(x_test)
log_loss_opt = metrics.log_loss(y_test, y_predict)
f"Intel® extension for Scikit-learn Log Loss: {log_loss_opt} s"

'Intel® extension for Scikit-learn Log Loss: 0.6872218096013292 s'

In [21]:
#unpatch the oneapi scikit learn
#unpatch the oneapi scikit learn 
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [22]:
from sklearn.linear_model import LogisticRegression

start = timer()
classifier = LogisticRegression(**params).fit(x_train, y_train)
train_unpatched = timer() - start
f"Original Scikit-learn time: {train_unpatched:.2f} s"

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


'Original Scikit-learn time: 3.37 s'

In [23]:
y_predict = classifier.predict_proba(x_test)
log_loss_original = metrics.log_loss(y_test, y_predict)
f"Original Scikit-learn Log Loss: {log_loss_original} s"

'Original Scikit-learn Log Loss: 0.6829430001616089 s'

In [24]:
HTML(f"<h3>Compare Log Loss metric of patched Scikit-learn and original</h3>"
     f"Log Loss metric of patched Scikit-learn: {log_loss_opt} <br>"
     f"Log Loss metric of unpatched Scikit-learn: {log_loss_original} <br>"
     f"Metrics ratio: {log_loss_opt/log_loss_original} <br>"
     f"<h3>With Scikit-learn-intelex patching you can:</h3>"
     f"<ul>"
     f"<li>Use your Scikit-learn code for training and prediction with minimal changes (a couple of lines of code);</li>"
     f"<li>Fast execution training and prediction of Scikit-learn models;</li>"
     f"<li>Get the similar quality</li>"
     f"<li>Get speedup in <strong>{(train_unpatched/train_patched):.1f}</strong> times.</li>"
     f"</ul>")